In [5]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer

from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth


Lets try and do a few things
- are different countries described differently?
    - how about different regions?
- rating by region
- common words by variety
- bucket points into bands of ten, most common words in each bucket
- most common words by price
- most common words low value (points/price < low)
- most common words high value
- map of top regions
- map of most expensive regions


EXTRA - 
can I generate a fake review, region and winery for NLP?

In [2]:
df = pd.read_csv('/Users/jackohagan/datascience/wine/winemag-data_first150k.csv')

In [3]:
df.head()


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [4]:
df['area'] = df['region_1'] + ', ' + df['province'] 

In [5]:
df.head()



,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery,area
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,"Napa Valley, California"
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,"Toro, Northern Spain"
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,"Knights Valley, California"
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,"Willamette Valley, Oregon"
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,"Bandol, Provence"


In [6]:

unq_address = df['area'].unique()

data = pd.DataFrame()

data['unq_adress'] = unq_address


def findGeocode(city):
    try: geolocator=Nominatim(user_agent="myemail_address@gmail.com")
    except GeocoderTimedOut:
        sleep(10)
        return findGeocode(city)
    return geolocator.geocode(city)

data["loc"] = data['unq_adress'].apply(geolocator.geocode)
data["point"]= data["loc"].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat', 'lon', 'altitude']] = pd.DataFrame(df['point'].to_list(), index=df.index)

NameError: name 'geolocator' is not defined

In [7]:
##filter stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')

stop.append('wine')

# Exclude stopwords 
df['description_cleaned'] =  df['country'].astype(str) + ', ' + df['description'].str.lower() 
df['description_cleaned'] = df['description_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))




In [9]:

#remove punctuation
df["description_cleaned"] = df["description_cleaned"].str.replace('[^\w\s]','')

/var/folders/_9/dygt4j8d1s1197cm8ymg58cm0000gn/T/ipykernel_2599/1609771943.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["description_cleaned"] = df["description_cleaned"].str.replace('[^\w\s]','')


In [10]:
## tokenize words (turn into list)


df['tokenized_sents']  = df.apply(lambda row: nltk.word_tokenize(row['description_cleaned']), axis=1)




In [11]:
##lemmatize the tokens - get the 'root' of each word

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

df['lemmatied_description'] = df['tokenized_sents'].apply(lemmatize_text)

In [15]:
##add extra columns, bin into 
##points / price - value
df['value'] = df['points']/df['price']
df['price_bucket'] = pd.qcut(df['price'],20,labels=["low", "lowmid", "lowhigh",'midlow','midmid','midhigh','highlow','highmid','highhigh','10','11','12','13','14','15','16','17','18','19','high'])
df['points_bucket'] = pd.cut(df['points'],5, labels=["low", "midlow", "mid",'midhigh','high'])
df['value_bucket'] = pd.qcut(df['value'],20)
df.to_csv('tidieddf.csv')

In [13]:
df= pd.read_csv('tidieddf.csv')

In [129]:
from mlxtend.preprocessing import TransactionEncoder

def associationfunction(df, column):
    


    # combine target and description into one list
    df['description_cleaned'] =  df[column].astype(str) + ', ' + df['description'].str.lower() 
    # Exclude stopwords 
    df['description_cleaned'] = df['description_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    #remove punctuation
    df["description_cleaned"] = df["description_cleaned"].str.replace('[^\w\s]','')
    
    

    ## tokenize words (turn into list)
    df['tokenized_sents']  = df.apply(lambda row: nltk.word_tokenize(row['description_cleaned']), axis=1)


    ##lemmatize the tokens - get the 'root' of each word
    df['lemmatied_description'] = df['tokenized_sents'].apply(lemmatize_text)

    
    
    ##associations 
    a_list = df['lemmatied_description'].tolist()
    ##enconde 1 hot
    te = TransactionEncoder()
    te_ary = te.fit(a_list).transform(a_list)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    
    frequent_itemsets = fpgrowth(df, min_support=0.001, use_colnames=True)
    frequent = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.05)
    
    return frequent

In [17]:
countryass = associationfunction(df=df, column='country')

/var/folders/_9/dygt4j8d1s1197cm8ymg58cm0000gn/T/ipykernel_1161/3287601297.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df["description_cleaned"] = df["description_cleaned"].str.replace('[^\w\s]','')


In [18]:
countryass["antecedent_len"] = countryass["antecedents"].apply(lambda x: len(x))
countryass["consequent_len"] = countryass["consequents"].apply(lambda x: len(x))

countryass = countryass[ (countryass['antecedent_len'] == 1) &
       (countryass['consequent_len'] ==1) ]

In [19]:
countries = df['country'].unique()


In [20]:
##write file to csv
countryass.to_csv('country_similarties.csv')

In [21]:
##for varietys
countryass = associationfunction(df=df, column='variety')

/var/folders/_9/dygt4j8d1s1197cm8ymg58cm0000gn/T/ipykernel_1161/3287601297.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df["description_cleaned"] = df["description_cleaned"].str.replace('[^\w\s]','')


In [22]:
countryass["antecedent_len"] = countryass["antecedents"].apply(lambda x: len(x))
countryass["consequent_len"] = countryass["consequents"].apply(lambda x: len(x))

countryass = countryass[ (countryass['antecedent_len'] == 1) &
       (countryass['consequent_len'] ==1) ]

##write file to csv
countryass.to_csv('variety_similarties.csv')

In [8]:
##for points
countryass= pd.read_csv('variety_similarties.csv')
countryass = associationfunction(df=df, column='points_bucket')

NameError: name 'associationfunction' is not defined

In [24]:
countryass["antecedent_len"] = countryass["antecedents"].apply(lambda x: len(x))
countryass["consequent_len"] = countryass["consequents"].apply(lambda x: len(x))

countryass = countryass[ (countryass['antecedent_len'] == 1) &
       (countryass['consequent_len'] ==1) ]

##write file to csv
countryass.to_csv('points_similarties.csv')
#countryass[countryass['antecedents']  == {'low'}].sort_values('lift',ascending=False)

In [ ]:
##for points
countryass= pd.read_csv('points_similarties.csv')
countryass = associationfunction(df=df, column='price_bucket')

In [26]:
countryass["antecedent_len"] = countryass["antecedents"].apply(lambda x: len(x))
countryass["consequent_len"] = countryass["consequents"].apply(lambda x: len(x))

countryass = countryass[ (countryass['antecedent_len'] == 1) &
       (countryass['consequent_len'] ==1) ]

##write file to csv
countryass.to_csv('price_similarties.csv')
#countryass[countryass['antecedents']  == {'low'}].sort_values('lift',ascending=False)

In [16]:
countryass=pd.read_csv('price_similarties.csv')
countryass[countryass['antecedents']  == {'high'}].sort_values('lift',ascending=False)

,Unnamed: 0,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len


In [18]:
countryass.sort_values('lift',ascending=False)

,Unnamed: 0,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
40434,840280,frozenset({'vinho'}),frozenset({'verde'}),0.001398,0.001418,0.001371,0.981043,691.910130,0.001370,52.675207,1,1
40435,840281,frozenset({'verde'}),frozenset({'vinho'}),0.001418,0.001398,0.001371,0.967290,691.910130,0.001370,30.528690,1,1
38912,835940,frozenset({'touriga'}),frozenset({'nacional'}),0.001696,0.001570,0.001564,0.921875,587.082674,0.001561,12.779901,1,1
38913,835941,frozenset({'nacional'}),frozenset({'touriga'}),0.001570,0.001696,0.001564,0.995781,587.082674,0.001561,236.598012,1,1
40917,845686,frozenset({'record'}),frozenset({'track'}),0.001378,0.001405,0.001034,0.750000,533.950472,0.001032,3.994382,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,138921,frozenset({'orange'}),frozenset({'tannin'}),0.033698,0.215756,0.001955,0.058002,0.268834,-0.005316,0.832533,1,1
20307,565721,frozenset({'aftertaste'}),frozenset({'finish'}),0.013934,0.284629,0.001047,0.075131,0.263961,-0.002919,0.773484,1,1
20604,566961,frozenset({'stone'}),frozenset({'tannin'}),0.024945,0.215756,0.001385,0.055511,0.257288,-0.003997,0.830337,1,1
9298,304791,frozenset({'almond'}),frozenset({'tannin'}),0.025369,0.215756,0.001345,0.053016,0.245725,-0.004129,0.828150,1,1


In [1]:
import numpy as np
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, BatchNormalization,Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


Init Plugin
Init Graph Optimizer
Init Kernel


In [13]:
stop =[]

df= pd.read_csv('tidieddf.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,country,description,designation,points,price,province,region_1,region_2,variety,winery,area,description_cleaned,tokenized_sents,lemmatied_description,value,price_bucket,points_bucket,value_bucket
0,0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,"Napa Valley, California",US tremendous 100 varietal hails oakville aged...,"['US', 'tremendous', '100', 'varietal', 'hails...","['US', 'tremendous', '100', 'varietal', 'hail'...",0.408511,high,midhigh,"(0.041999999999999996, 1.13]"
1,1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,"Toro, Northern Spain",Spain ripe aromas fig blackberry cassis soften...,"['Spain', 'ripe', 'aromas', 'fig', 'blackberry...","['Spain', 'ripe', 'aroma', 'fig', 'blackberry'...",0.872727,high,midhigh,"(0.041999999999999996, 1.13]"
2,2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,"Knights Valley, California",US mac watson honors memory made mother tremen...,"['US', 'mac', 'watson', 'honors', 'memory', 'm...","['US', 'mac', 'watson', 'honor', 'memory', 'ma...",1.066667,high,midhigh,"(0.041999999999999996, 1.13]"
3,3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,"Willamette Valley, Oregon",US spent 20 months 30 new french oak incorpora...,"['US', 'spent', '20', 'months', '30', 'new', '...","['US', 'spent', '20', 'month', '30', 'new', 'f...",1.476923,19,midhigh,"(1.13, 1.483]"
4,4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,"Bandol, Provence",France top la bégude named highest point viney...,"['France', 'top', 'la', 'bégude', 'named', 'hi...","['France', 'top', 'la', 'bégude', 'named', 'hi...",1.439394,19,midhigh,"(1.13, 1.483]"


In [78]:
df= pd.read_csv('tidieddf.csv')

# Exclude stopwords 
df['description_cleaned'] =  df['description'].str.lower().astype(str)
## tokenize words (turn into list)

df["description_cleaned"] = df["description_cleaned"].str.replace('[^\w\s]','')

#df['description_cleaned'] = df['description_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

df['test'] = df.apply(lambda row: nltk.word_tokenize(row['description_cleaned']), axis=1)

df['tokenized_sents'] = df['description_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))




/var/folders/_9/dygt4j8d1s1197cm8ymg58cm0000gn/T/ipykernel_1135/3304987619.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df["description_cleaned"] = df["description_cleaned"].str.replace('[^\w\s]','')


In [ ]:
df.to_csv('testdata.csv')
df = pd.read_csv('testdata.csv')

In [79]:
df.head()

,Unnamed: 0,Unnamed: 0.1,country,description,designation,points,price,province,region_1,region_2,...,winery,area,description_cleaned,tokenized_sents,lemmatied_description,value,price_bucket,points_bucket,value_bucket,test
0,0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,...,Heitz,"Napa Valley, California",this tremendous 100 varietal wine hails from o...,this tremendous 100 varietal wine hails from o...,"['US', 'tremendous', '100', 'varietal', 'hail'...",0.408511,high,midhigh,"(0.041999999999999996, 1.13]","[this, tremendous, 100, varietal, wine, hails,..."
1,1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,...,Bodega Carmen Rodríguez,"Toro, Northern Spain",ripe aromas of fig blackberry and cassis are s...,ripe aromas of fig blackberry and cassis are s...,"['Spain', 'ripe', 'aroma', 'fig', 'blackberry'...",0.872727,high,midhigh,"(0.041999999999999996, 1.13]","[ripe, aromas, of, fig, blackberry, and, cassi..."
2,2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,...,Macauley,"Knights Valley, California",mac watson honors the memory of a wine once ma...,mac watson honors the memory of a wine once ma...,"['US', 'mac', 'watson', 'honor', 'memory', 'ma...",1.066667,high,midhigh,"(0.041999999999999996, 1.13]","[mac, watson, honors, the, memory, of, a, wine..."
3,3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,...,Ponzi,"Willamette Valley, Oregon",this spent 20 months in 30 new french oak and ...,this spent 20 months in 30 new french oak and ...,"['US', 'spent', '20', 'month', '30', 'new', 'f...",1.476923,19,midhigh,"(1.13, 1.483]","[this, spent, 20, months, in, 30, new, french,..."
4,4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,...,Domaine de la Bégude,"Bandol, Provence",this is the top wine from la bégude named afte...,this is the top wine from la bégude named afte...,"['France', 'top', 'la', 'bégude', 'named', 'hi...",1.439394,19,midhigh,"(1.13, 1.483]","[this, is, the, top, wine, from, la, bégude, n..."


In [80]:
##train on 10000 samples to speed up
df= df.sample(n=10000, replace=False).reset_index()

In [81]:
df['tokenized_sets'] = df['test']
df.head()

,index,Unnamed: 0,Unnamed: 0.1,country,description,designation,points,price,province,region_1,...,area,description_cleaned,tokenized_sents,lemmatied_description,value,price_bucket,points_bucket,value_bucket,test,tokenized_sets
0,124182,124182,124182,Italy,This thickly extracted Montepulciano d'Abruzzo...,Valle d'Oro,89,NaN,Central Italy,Montepulciano d'Abruzzo,...,"Montepulciano d'Abruzzo, Central Italy",this thickly extracted montepulciano dabruzzo ...,this thickly extracted montepulciano dabruzzo ...,"['Italy', 'thickly', 'extracted', 'montepulcia...",NaN,NaN,mid,NaN,"[this, thickly, extracted, montepulciano, dabr...","[this, thickly, extracted, montepulciano, dabr..."
1,102837,102837,102837,Portugal,"This is a lightly wood-aged wine, with melon a...",Quinta do Encontro Q do E,89,NaN,Bairrada,NaN,...,NaN,this is a lightly woodaged wine with melon and...,this is a lightly woodaged wine with melon and...,"['Portugal', 'lightly', 'woodaged', 'wine', 'm...",NaN,NaN,mid,NaN,"[this, is, a, lightly, woodaged, wine, with, m...","[this, is, a, lightly, woodaged, wine, with, m..."
2,79171,79171,79171,France,"This has a warm, rounded character, with press...",Vaillons Premier Cru,91,30.0,Burgundy,Chablis,...,"Chablis, Burgundy",this has a warm rounded character with pressin...,this has a warm rounded character with pressin...,"['France', 'warm', 'rounded', 'character', 'pr...",3.033333,13,mid,"(2.8, 3.036]","[this, has, a, warm, rounded, character, with,...","[this, has, a, warm, rounded, character, with,..."
3,110773,110773,110773,Portugal,"A smooth, toasty wine, with distinct caramel a...",Vinhas Velhas,89,20.0,Beiras,NaN,...,NaN,a smooth toasty wine with distinct caramel and...,a smooth toasty wine with distinct caramel and...,"['Portugal', 'smooth', 'toasty', 'wine', 'dist...",4.450000,highmid,mid,"(4.35, 4.632]","[a, smooth, toasty, wine, with, distinct, cara...","[a, smooth, toasty, wine, with, distinct, cara..."
4,150608,150608,150608,US,"Old vine clone, with brambly, gamy notes throu...",Rockpile Road,90,19.0,California,Dry Creek Valley,...,"Dry Creek Valley, California",old vine clone with brambly gamy notes through...,old vine clone with brambly gamy notes through...,"['US', 'old', 'vine', 'clone', 'brambly', 'gam...",4.736842,highlow,mid,"(4.632, 5.0]","[old, vine, clone, with, brambly, gamy, notes,...","[old, vine, clone, with, brambly, gamy, notes,..."


In [82]:
##convert tokens to list and then numbers
processed_inputs = df['tokenized_sents']
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [83]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 10000
Total vocab: 9704


In [84]:
seq_length = 250
x_data = []
y_data = []

In [85]:
# loop through inputs, start at the beginning and go until we hit
# the final character we can create a sequence out of
for i in range(0, input_len - seq_length, 1):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = processed_inputs[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = processed_inputs[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

In [86]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 9750


In [87]:
X = np.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [88]:
y = np_utils.to_categorical(y_data)

In [94]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))

model.add(LSTM(256, return_sequences=True))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(LSTM(128))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dense(y.shape[1], activation='softmax'))

In [95]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [96]:
#It takes the model quite a while to train,
#and for this reason we'll save the weights and reload them when the training is finished.
#We'll set a checkpoint to save the weights to, and then make them the callbacks for our future model.

filepath = "model_weights_saved_valid.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [92]:
model.fit(X, y, epochs=30, batch_size=128, callbacks=desired_callbacks, validation_split=0.05)

Epoch 1/30


2021-12-12 17:38:43.012542: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-12 17:38:43.165424: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-12 17:38:44.082224: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-12 17:38:45.574770: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-12 17:38:46.525018: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-12 17:38:48.369449: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-12 17:38:50.195050: I tensorflow/core/grappler/optimizers/cust

73/73 [==============================] - ETA: 0s - loss: 9.1995

2021-12-12 17:39:33.562915: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-12 17:39:33.616109: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-12 17:39:34.487149: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-12 17:39:35.390395: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


73/73 [==============================] - 58s 655ms/step - loss: 9.1996 - val_loss: 9.3197

Epoch 00001: loss improved from inf to 9.20986, saving model to model_weights_saved_test.hdf5
Epoch 2/30
73/73 [==============================] - 36s 484ms/step - loss: 9.0714 - val_loss: 9.5953

Epoch 00002: loss improved from 9.20986 to 9.05256, saving model to model_weights_saved_test.hdf5
Epoch 3/30
73/73 [==============================] - 35s 480ms/step - loss: 8.7746 - val_loss: 11.8012

Epoch 00003: loss improved from 9.05256 to 8.76748, saving model to model_weights_saved_test.hdf5
Epoch 4/30
73/73 [==============================] - 34s 466ms/step - loss: 8.2209 - val_loss: 11.1170

Epoch 00004: loss improved from 8.76748 to 8.31235, saving model to model_weights_saved_test.hdf5
Epoch 5/30
73/73 [==============================] - 34s 468ms/step - loss: 7.8111 - val_loss: 12.4563

Epoch 00005: loss improved from 8.31235 to 7.96969, saving model to model_weights_saved_test.hdf5
Epoch 6/30
7

KeyboardInterrupt: 

In [97]:
filename = "model_weights_saved_valid.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

ValueError: Cannot assign to variable dense_5/kernel:0 due to variable shape (128, 9704) and value shape (128, 4929) are incompatible

In [98]:
#convert model output back to chars
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [100]:
num_to_char[3000]

'floral earthy funky and barnyardy with all the young exuberance that comes with jovenroble wines from rdd lots of color power tannins and extract along with young sweet chewy blackfruit flavors and tart acidity to the palate'

In [60]:
##random seed to set off model
num_to_char[2000]

'hugely dry tannic wine that is going to need several years aging but the structure and balance are there along with dark berry flavors mixed with a firm mineral character this is powerful stuff'

In [52]:
pattern = x_data[start]
pattern
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")



Random Seed:
" superripe in the modern style with blackberry cherry chocolate anise bacon and cedar flavors that are dry and complex however excessively high alcohol of 161 gives the wine a soft glyceriney sweetness and heat that detractfrom an estate vineyard off olivet lane in the heart of the santa rosa plain this wine is comparatively intense and meaty compared to some of its counterparts from the cooler sebastopol hills earthy mushroom and a tart tangy note of orange give it a lift along with a light layering of freshcut rose before finishing in richer dark chocolatecreamy fresh white fruit and toast on the nose hint at a fullness that the wine does not quite deliver on the palate but overall the fruit is good this tends toward tart and needs the balance of more oak to cut the edgefaiveleys general firm style of red wine is a great advantage in this simple burgundy this wine has structure red berry acidity ripe tannins and a lively vivid flavor to finish there is plenty of ripe ac

In [197]:
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = num_to_char[index]

    sys.stdout.write(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]

2021-12-12 00:42:38.107470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-12 00:42:38.158937: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-12 00:42:38.377442: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-12 00:42:38.604323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


good flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice

good flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice

good flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice

good flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice

...

good flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice

good flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice

good flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice

good flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice and toast in this classic blend of chardonnay pinot noir and pinot meunier its dry but rough in the mouth with a scoury finishgood flavors of citrus peach spice

KeyboardInterrupt: 

In [ ]:
##need to generate more based on seed word - how do i do this?

from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
from time import sleep
from random import randint

# declare an empty list to store
# latitude and longitude of values
# of city column
longitude = []
latitude = []


# function to find the coordinate
# of a given city
def findGeocode(city):

# try and catch is used to overcome
# the exception thrown by geolocator
# using geocodertimedout
    try:

    # Specify the user_agent as your
    # app name it should not be none
    
        user_agent = 'user_me_{}'.format(randint(10000,99999))

        geolocator = Nominatim(user_agent=user_agent)

        return geolocator.geocode(city)

    except GeocoderTimedOut:
        
            logging.info('TIMED OUT: GeocoderTimedOut: Retrying...')
            sleep(randint(1*100,sleep_sec*100)/100)
            
    return geolocator.geocode(city)

    # each value from city column
    # will be fetched and sent to
    # function find_geocode
for i in (unq_address):

    if findGeocode(i) != None:

        loc = findGeocode(i)

    # coordinates returned from
    # function is stored into
        # two separate list
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)

        # if coordinate for a city not
        # found, insert "NaN" indicating
        # missing value
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)



In [102]:
!jupyter nbconvert --to script Wineanalysis.ipynb

[NbConvertApp] Converting notebook Wineanalysis.ipynb to script
[NbConvertApp] Writing 12441 bytes to Wineanalysis.py


In [112]:
pd.DataFrame.from_dict(num_to_char,orient='index', columns=['review']).to_csv('generatedreviews.csv')